In [2]:
import pandas as pd
import numpy as np
import csv

In [3]:
def prep_echo(path, outname):
    #data = pd.read_csv('echosoundertest.txt',sep=" ",header = None)
    df = pd.read_fwf(path)
    # Add 0 in front of numbers that are less than 10
    df['MO'] = df['MO'].apply(lambda x: str(x) if x > 9 else str(x).zfill(2))
    # Create dataid to match sensors, set as index
    df['dataid']=df['YEAR'].astype(str)+'-'+df['MO']+'-'+df['DA'].astype(str)+'T'+df['UTC']
    df.set_index('dataid')
    # Cast to datetime, round to nearest half hour
    df.dataid = pd.to_datetime(df.dataid, format='%Y-%m-%d %H:%M:%S')
    df.dataid = df['dataid'].dt.round('30min') 
    # Drop unnecessary columns
    df = df.drop(['BDMIN', 'BDMAX', 'OBJECT', 'UPINLM', 'PDMIN', 'PDMAX', 'LOG1', 'LOG2', 'CH', 'YEAR', 'MO', 'DA', 'UTC', 'Me/Pl' ], axis=1)
    df.to_csv(f'{outname}.csv', index=False)
    return df

In [ ]:
df = prep_echo('leg2_data.txt', 'leg2_echo_prepped')

In [ ]:
def iden (row):
    month = str(row[1]) if row[1] > 9 else '0'+str(row[1])
    day = str(row[2]) if row[2] > 9 else '0'+str(row[2])
    
    return str(row[0])+'-'+month+'-'+day+'T'+str(row[3])

In [ ]:
print(arr)

In [ ]:
df

In [4]:
import xarray as xr

ds = xr.open_dataset('GEBCO_13_Jul_2022_67c854092a40/gebco_2022_n60.7495_s-46.3821_w-100.2326_e14.0639.nc')
dsf = ds.to_dataframe()

dsf = dsf.reset_index()


In [5]:
def add_seafloor_depth(echo, sf):
    for index, row in echo.iterrows():
        closest_depth = find_closest(sf, row['LATITUDE'], row['LONGITUD'])
        echo.at[index,'sf_depth'] = closest_depth
        
    return echo

In [14]:
df = pd.read_csv('leg4_echo_prepped.csv')
df.dataid = pd.to_datetime(df['dataid'])
df['LATITUDE'] = np.where((df['LATITUDE']>=10) & (df['dataid'] >= '2022-02-13'), -df['LATITUDE'], df['LATITUDE'])
 
uniq = df[['LATITUDE', 'LONGITUD']].value_counts().reset_index(name='sf_depth')

,LATITUDE,LONGITUD,sf_depth
count,592.000000,592.000000,592.0
mean,1.109639,-32.079918,73.0
std,15.342531,3.232202,0.0
min,-23.194540,-40.661920,73.0
25%,-13.052125,-34.221292,73.0
50%,3.178735,-30.901220,73.0
75%,14.734540,-29.273235,73.0
max,26.689290,-28.439590,73.0


In [9]:
df_w_depth = add_seafloor_depth(uniq, dsf_4)

In [7]:
def find_closest(df, lat, lon):
    dist = (df['lat'] - lat).abs() + (df['lon'] - lon).abs()
    closest = df.loc[dist.idxmin()]
    return closest['elevation']

In [8]:
dsf_4 = dsf.loc[(dsf['lat'] >= -23.194540) & (dsf['lat'] <= 26.689290)]

In [10]:
new_df = pd.merge(df, df_w_depth,  how='left', on=['LATITUDE', 'LONGITUD'])

In [11]:
new_df

,LATITUDE,LONGITUD,PDMEAN,TOTAL,dataid,sf_depth
0,26.68929,-29.05914,7.5,0.0000,2022-01-29 07:30:00,-5264
1,26.68929,-29.05914,15.0,4.3472,2022-01-29 07:30:00,-5264
2,26.68929,-29.05914,25.0,10.4678,2022-01-29 07:30:00,-5264
3,26.68929,-29.05914,35.0,12.3473,2022-01-29 07:30:00,-5264
4,26.68929,-29.05914,45.0,14.3287,2022-01-29 07:30:00,-5264
...,...,...,...,...,...,...
43211,-23.19454,-40.66192,685.0,35.0402,2022-02-19 21:00:00,-1894
43212,-23.19454,-40.66192,695.0,45.1827,2022-02-19 21:00:00,-1894
43213,-23.19454,-40.66192,705.0,351.3631,2022-02-19 21:00:00,-1894
43214,-23.19454,-40.66192,715.0,42.6268,2022-02-19 21:00:00,-1894


In [12]:
new_df.to_csv('leg4_echo_prepped.csv', index=False)